In [35]:
from modlamp.descriptors import GlobalDescriptor
import numpy as np

In [36]:
codon_table = {
    'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
    'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
    'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
    'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
    'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
    'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
    'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
    'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
    'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
    'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
    'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
    'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
    'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
    'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
    'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
    'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
}


In [37]:
def geneToProtein(dna_seqs, verbose=True):
    tot = 0
    ans = 0
    global codon_table
    p_seqs = []
    for dna_seq in dna_seqs:
        p_seq = ""
        if dna_seq[0:3] != 'ATG':
            tot = tot + 1
            #if verbose: print("Not valid gene (no ATG)")
            continue
        for i in range(3, len(dna_seq), 3):
            codon = dna_seq[i:i+3]
            try:
                aa = codon_table[codon]
                p_seq += aa
                if aa == '_': break
            except:
                #if verbose: print("Error! Invalid Codon {} in {}".format(codon, dna_seq))
                break
        if len(p_seq) <= 2: #needs to have stop codon and be of length greater than 2
            tot = tot + 1
            #if verbose: print("Error! Protein too short.")
        elif p_seq[-1] != '_':
            tot = tot + 1
            #if verbose: print("Error! No valid stop codon.")
        else:
            ans = ans + 1
            p_seqs += [p_seq[:-1]]
    return p_seqs, tot, ans

In [38]:
def average(list, cnt, fpp):
    tot = 0
    for i in list:
        tot = tot + i
    ave = tot / cnt
    exceed = 0
    under = 0
    maxx = 0
    minn = 10000000000
    for i in list:
        if i > ave and i - ave > exceed:
            exceed = i - ave
        elif i < ave and ave - i > under:
            under = ave - i
        if i > maxx:
            maxx = i;
        if i < minn:
            minn = i;
    print("averange:",ave, file = fpp)
    print("maxnum:",maxx, file = fpp)
    print("minnum:",minn, file = fpp)
    print("variance:",np.sqrt(np.var(list)), file = fpp)

In [49]:
# dna_seqs = []
# for ii in range(5):
#     if ii >= 0:
#         i = ii + 1
#         sampled_path = "./amp_data_before_fbgan/"
#         text = sampled_path + "sample_val_{}.txt".format(i)
#         with open(text, "r") as f:
#             for line in f.readlines():
#                 line = line.strip().split()
#                 dna_seq = line[0]
#                 dna_seqs.append(dna_seq)
# p_seqs, tot, ans = geneToProtein(dna_seqs)
# print(p_seqs, tot, ans)


for tem in range(1):
    # T = tem + 1
    T = 4
    print(T)
    
    sample_path = "./data_now/fbgan_{}/".format(T)
    dna_seqs = []
    for ii in range(150):
        if ii >= 59 and ii < 140:
            i = ii + 1
            
            #text = sample_path + "sampled_{}_preds.txt".format(i)
            text = sample_path + "sample_val_{}.txt".format(i)
            with open(text, "r") as f:
                for line in f.readlines():
                    line = line.strip().split()
                    dna_seq = line[0]
                    dna_seqs.append(dna_seq)
    p_seqs_s, tot, ans = geneToProtein(dna_seqs)
    p_seqs =[]
    for p_seq in p_seqs_s:
        #if len(p_seq) >= 30 and len(p_seq) <= 40:
        p_seqs.append(p_seq)
    fpp = open(sample_path + "pre/total.txt","w+")
    print(tot, ans, file = fpp)


    # Sequence Length
    desc = GlobalDescriptor(p_seqs)
    desc.length()
    fp = open(sample_path + "pre/length.txt", "w+")
    cnt = 0
    list = []
    for i in desc.descriptor:
        list.append(i[0])
        cnt = cnt + 1
        print(int(i[0]), file = fp)
    print("Sequence Length", file = fpp)
    average(list, cnt, fpp)

    #Molecular Formula
    desc.calculate_MW(amide = True)
    fp = open(sample_path + "pre/Molecular_Formula.txt", "w+")
    cnt = 0
    list = []
    for i in desc.descriptor:
        list.append(i[0])
        cnt = cnt + 1
        print(i[0], file = fp)
    print("Molecular Formula", file = fpp)
    average(list, cnt, fpp)

    #Sequence Charge
    desc.calculate_charge(ph = 7.0, amide = True)
    fp = open(sample_path + "pre/Sequence_Charge.txt", "w+")
    cnt = 0
    list = []
    for i in desc.descriptor:
        list.append(i[0])
        cnt = cnt + 1
        print(i[0], file = fp)
    print("Sequence Charge", file = fpp)
    average(list, cnt, fpp)

    #Charge Density
    desc.charge_density(ph = 7.0, amide = True)
    fp = open(sample_path + "pre/Charge_Density.txt", "w+")
    cnt = 0
    list = []
    for i in desc.descriptor:
        list.append(i[0])
        cnt = cnt + 1
        print(i[0], file = fp)
    print("Charge Density", file = fpp)
    average(list, cnt, fpp)

    #Isoelectric_point
    desc.isoelectric_point()
    fp = open(sample_path + "pre/Isoelectric_point.txt", "w+")
    cnt = 0
    list = []
    for i in desc.descriptor:
        list.append(i[0])
        cnt = cnt + 1
        print(i[0], file = fp)
    print("Isoelectric_point", file = fpp)
    average(list, cnt, fpp)

    #instability_index
    desc.instability_index()
    fp = open(sample_path + "pre/Instability_index.txt", "w+")
    cnt = 0
    list = []
    for i in desc.descriptor:
        list.append(i[0])
        cnt = cnt + 1
        print(i[0], file = fp)
    print("instability_index", file = fpp)
    average(list, cnt, fpp)

    #aromaticity
    desc.aromaticity()
    fp = open(sample_path + "pre/Aromaticity.txt", "w+")
    cnt = 0
    list = []
    for i in desc.descriptor:
        list.append(i[0])
        cnt = cnt + 1
        print(i[0], file = fp)
    print("aromaticity", file = fpp)
    average(list, cnt, fpp)

    #Aliphatic_index
    desc.aliphatic_index()
    fp = open(sample_path + "pre/Aliphatic_index.txt", "w+")
    cnt = 0
    list = []
    for i in desc.descriptor:
        list.append(i[0])
        cnt = cnt + 1
        print(i[0], file = fp)
    print("Aliphatic_index", file = fpp)
    average(list, cnt, fpp)

    #Boman_index
    desc.boman_index()
    fp = open(sample_path + "pre/Boman_index.txt", "w+")
    cnt = 0
    list = []
    for i in desc.descriptor:
        list.append(i[0])
        cnt = cnt + 1
        print(i[0], file = fp)
    print("Boman_index", file = fpp)
    average(list, cnt, fpp)

    #hydrophobic_ratio
    desc.hydrophobic_ratio()
    fp = open(sample_path + "pre/Hydrophobic_ratio.txt", "w+")
    cnt = 0
    list = []
    for i in desc.descriptor:
        list.append(i[0])
        cnt = cnt + 1
        print(i[0], file = fp)
    print("hydrophobic_ratio", file = fpp)
    average(list, cnt, fpp)

    #calculate_all
    desc.calculate_all()
    fp = open(sample_path + "pre/all_physiochemical_properties.txt", "w+")
    print(desc.featurenames, file = fp)
    for i in desc.descriptor:
        print(i, file = fp)

print("done")

4
done
